In [0]:
#install the libraries
!pip install selenium
!pip install Pillow
!pip install plotly
!pip install dill
!pip install git+git://github.com/scikit-optimize/scikit-optimize.git # PyPI doesnt have the latest version of scikit optimize that has checkpointsaver
!pip install -U git+https://github.com/mattiavarile/efficientnet # this specific efficientnet is coded with tensorflow keras and not keras
#!pip install -U git+https://github.com/keras-team/keras-tuner # Still work in progress but promising
!pip install tensorflow==1.13.1 # tf 1.14 has a bug where you can't compile a model with weights, wich is needed for transfer learning so 1.13.1 is the latest working version

     |████████████████████████████████| 911kB 1.4MB/s 
  Cloning git://github.com/scikit-optimize/scikit-optimize.git to /tmp/pip-req-build-jdb_xpja
  Running command git clone -q git://github.com/scikit-optimize/scikit-optimize.git /tmp/pip-req-build-jdb_xpja
  Created wheel for scikit-optimize: filename=scikit_optimize-0.6+3.g6740876-py2.py3-none-any.whl size=75044 sha256=16f48411b560c0fb9d26895481a8784fea3b6c13909bf5be1b902dbec99fd4ee
  Stored in directory: /tmp/pip-ephem-wheel-cache-8yqvb4xu/wheels/87/8f/00/55cc0281663e6264de266909eb75d07afb935bde4e34d12c3e
Successfully built scikit-optimize
  Cloning https://github.com/mattiavarile/efficientnet to /tmp/pip-req-build-4179bluc
  Running command git clone -q https://github.com/mattiavarile/efficientnet /tmp/pip-req-build-4179bluc
  Created wheel for efficientnet: filename=efficientnet-0.0.3-cp36-none-any.whl size=11753 sha256=bbc991123d147bf3befcb96cdcadb65438723fecc4ea3881bb04915af6765d13
  Stored in directory: /tmp/pip-ephem-wheel-

In [0]:
# import librairies
from google import colab
import json
import tensorflow as tf
import tensorboard as tb
from importlib import reload

print("Tensorflow version " + tf.__version__)

# Choose the bucket name and project
BUCKET = 'gs://ai-decathlon-canada'

# PROJECT = 'decathon_classes_1_shard'
# PROJECT = 'decathon_classes_16_train_shard'
# PROJECT = 'decathon_classes_32_train_shard'
# PROJECT = 'parent_sport_classificationV3'
# PROJECT = 'parent_sport_classificationV4'
# PROJECT = 'parent_sport_classificationV4_w_da'
# PROJECT = 'parent_sport_classificationV5_w_da'
PROJECT = 'parent_sport_classificationV6_w_da'
# PROJECT = 'hockey_community'
# PROJECT = 'all_sport_groups'

Tensorflow version 1.13.1


In [0]:
# get image classification repo
%cd /content/
if tf.gfile.Exists('./image-classification'):
  tf.gfile.DeleteRecursively('./image-classification')
!git clone --branch tpu_bucket_dev https://github.com/decathloncanada/image-classification.git
!ls

/content
Cloning into 'image-classification'...
remote: Enumerating objects: 182, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 1638 (delta 126), reused 147 (delta 95), pack-reused 1456
Receiving objects: 100% (1638/1638), 241.93 MiB | 39.77 MiB/s, done.
Resolving deltas: 100% (374/374), done.
image-classification  sample_data


In [0]:
# google authentification
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
import pandas as pd
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 1.4MB/s 

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
# Get one of the image dataset from the drive
%cd /content/image-classification/
!ls
#step 4: get the file ID (you can get it from the sharing link - obtenir le lien partagé...)
#file_id = '1UDAlCGH72TdkNjVq6nvJqDStzR9L5o-P' #hockey dataset from google images
#file_id = '1bbeVpJwHpyUMOE3P11f_71ntTTW4jX_T' #hockey dataset from HC
#file_id = '12r-vS0TwCpwiAnHYT0RQ-EUzYuXogK8W' #hockey brand dataset from sidelineswap
#file_id = '1BMKeo9M_dZAVOEc0nLxKVOBp8a5r_CXl' #sport dataset
#file_id = '1S5-wkB2G_1XlefifTXKN0468_YqzFPsV' #smaller sport group dataset
#file_id = '1oVdkRyQGA6td9bwzU0G6Jmyu0Bco_SJn' #all sport groups dataset
#file_id = '1GMf_KD699pKm2wJgY-fikWlYuIYwVjIG' #all parents sport category
#file_id = '15fpLCGKCG_TA_ixuNJIt8oP4KTaQQUth' #all parents sport category V2
#file_id = '1EQ3UhdTP-3jYOeRW4O-kUK-_7s34BzKk' #all parents sport category V3
#file_id = '1iN3_sEde_gLaWNxesFPzjmEOOUcAUQto' #all parents sport category V4
#file_id = '1OjB-Kftb3F0C602bbkweAV-GFAygKum-' #all parents sport category V4 with data augmentation
#file_id = '1FL3vNOI7QQHMAKsX7FThcudc8zmu8bno' #all parents sport category V5 with data augmentation
file_id = '1J-0wsKEwGRSQd7oO63kRAL_3ZZtS5Ti7' #all parents sport category V6 with data augmentation
#file_id = '1EEp5CtJxrrScfRiIp8xN3Chk_YeB67T7' # decathlon classes

#step 5: extract the file to the data folder (with name "name_folder")
name_folder = 'image_dataset'
%cd data/
fileId = drive.CreateFile({'id': file_id}) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])
# 7z is required files larger than 4 Gb
fileId.GetContentFile(name_folder + '.7z')  # Save Drive file as a local file
!ls
#step 6: unzip the folder and delete zip
!p7zip -d 'image_dataset.7z'  && rm 'image_dataset.7z'

/content/image-classification
app.py	debug.log  main.py    README.md		src   utils
data	LICENSE    notebooks  requirements.txt	test
/content/image-classification/data
all_sports_parent_groupV6_w_da.7z
image_dataset.7z  searchterms.csv

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 568639750 bytes (543 MiB)

Extracting archive: image_dataset.7z
--
Path = image_dataset.7z
Type = 7z
Physical Size = 568639750
Headers Size = 867568
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      0% 460          0% 831 - image_dataset/train/aeromodeli . 8b2-43db-b00f-149695b770b4.jpg                                                                          

In [0]:
# Split dataset if needed
%cd /content/image-classification/
from utils import utils
utils.split_train(split=0.15)

In [0]:
# Verify that tpu or gpu is available and working
try:
  tpu = tf.contrib.cluster_resolver.TPUClusterResolver() # TPU detection
  use_tpu = True
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
except ValueError:
  if tf.test.is_built_with_cuda():
    print("Running on GPU")
    print(tf.test.gpu_device_name())
  else:
    print("Running on CPU")
  use_tpu = False
  tpu = None

Running on TPU  ['10.23.108.162:8470']


In [0]:
# Get the GCS directories
data_dir = os.path.join(os.path.join(BUCKET, 'data'), PROJECT)
work_dir = os.path.join(os.path.join(BUCKET, 'work'), PROJECT)
train_fn = os.path.join(data_dir, "train") # prefix for output file names
val_fn = os.path.join(data_dir, "val") # prefix for output file names
print('Data dir= '+data_dir)
print('Work dir= '+work_dir)
print('Train dir= '+train_fn)
print('Val dir= '+val_fn)

Data dir= gs://ai-decathlon-canada/data/decathon_classes_16_train_shard
Work dir= gs://ai-decathlon-canada/work/decathon_classes_16_train_shard
Train dir= gs://ai-decathlon-canada/data/decathon_classes_16_train_shard/train
Val dir= gs://ai-decathlon-canada/data/decathon_classes_16_train_shard/val


In [0]:
# generate tfrecords and updload to gcs
parentdir = '/content/image-classification'
%cd /content/image-classification/src
!ls
import generate_tfrecords;reload(generate_tfrecords)
from generate_tfrecords import TfrecordsGenerator
generator = TfrecordsGenerator()

generator.convert_image_folder(img_folder=parentdir+'/data/image_dataset/train', ouput_folder=train_fn, shards=1)
print('tfrecords generated and saved at '+ train_fn)
generator.convert_image_folder(img_folder=parentdir+'/data/image_dataset/val', ouput_folder=val_fn, shards=1)
print('tfrecords generated and saved at '+ val_fn)

In [0]:
# delete and reupload a file if needed, if quick changes are needed
%cd /content/image-classification/test/
!ls

# remove a file
if tf.gfile.Exists('image_classifier_tfrecords.py'):
  tf.gfile.Remove('image_classifier_tfrecords.py')
!ls

# upload new version
colab.files.upload()
!ls

/content/image-classification/test
image_classifier_from_tensor_slices.py	__pycache__
image_classifier_from_tensor_slices.py	__pycache__


Saving image_classifier_tfrecords.py to image_classifier_tfrecords.py
image_classifier_from_tensor_slices.py	__pycache__
image_classifier_tfrecords.py


In [0]:
# Training
import image_classifier_tfrecords;reload(image_classifier_tfrecords)
from image_classifier_tfrecords import ImageClassifier
classifier = ImageClassifier(tfrecords_folder=data_dir, use_TPU=use_tpu, 
                              batch_size=64, transfer_model='B5',
                              legacy=True)  # legacy is true because we are using pre-tf 1.14/2.0 version
classifier.fit(epochs=3, learning_rate=5e-4, l2_lambda=1e-3, activation='swish', dropout=0.9, save_model=False)

Classes (333) :
['1-PIECE SWIMSUI', '2 PIECES SWIMSU', 'ADAPTOR', 'ADDITIVE', 'AQUASHOES', 'ARM COVER', 'ARROW', 'BACKPACK', 'BADMINTON RACKE', 'BAG', 'BAIT', 'BALACLAVA', 'BALL', 'BANDANA', 'BASE LAYER', 'BASEBALL BAT', 'BASKET BALL', 'BATHROBE', 'BATTERY', 'BEANIE', 'BELT', 'BERMUDA SHORTS', 'BIB', 'BIKE', 'BIKE COMPUTER', 'BIKE GUARD', 'BIKE LIGHT', 'BIKE PANNIER', 'BIKE SADDLE', 'BIKINI', 'BIKINI BOTTOM', 'BIKINI TOP', 'BINDING', 'BINOCULARS', 'BMX', 'BOARDSHORTS', 'BOAT', 'BOILIES', 'BOMBETTE', 'BOOTS, BOOTIES', 'BOULE', 'BOW', 'BOXER SHORTS', 'BRA', 'BRAKE LEVER', 'BRAKE PAD', 'BRAKE PADS', 'BREECHES', 'BRIDLE', 'BRIEFS', 'BRUSH', 'CAMP STOVE', 'CANOE-KAYAK', 'CAP', 'CARRY BAG', 'CASSETTE', 'CEREAL BAR', 'CHAINSET', 'CHAIR', 'CHILD BIKE', 'CHOCOLATE', 'CITY BIKE', 'CLEATED SHOES', 'CLIMBING ROPE', 'COLLAR', 'COMMUNICATION S', 'COOKSET', 'COOLER BOX', 'CROPPED LEGGING', 'CROPPED TROUSER', 'CROSS TRAINER', 'CUP', 'CYCLE RACK', 'DART', 'DARTBOARD', 'DECOY', 'DEHYDRATED MEAL', 'DEODO

In [0]:
# Fitness
import image_classifier_tfrecords;reload(image_classifier_tfrecords)
from image_classifier_tfrecords import ImageClassifier
classifier = ImageClassifier(tfrecords_folder=data_dir, use_TPU=use_tpu, 
                                 batch_size=128, transfer_model='B3')
classifier.hyperparameter_optimization(num_iterations=28, n_random_starts=12)
%cd /content/image-classification/data/trained_models/
!ls
# download pickle files
colab.files.download("hyperparameters_dimensions.pickle")
colab.files.download("hyperparameters_search.pickle")

In [0]:
# upload hyperparameters
if not tf.gfile.Exists('/content/image-classification/data/trained_models'):
  tf.gfile.MkDir('/content/image-classification/data/trained_models')
%cd /content/image-classification/data/trained_models
f.files.upload()

In [0]:
#load the optimized hypermeters
import dill
with open('hyperparameters_dimensions.pickle', 'rb') as f:
    dimensions = dill.load(f)
with open('hyperparameters_search.pickle', 'rb') as f:
    sr = dill.load(f)
opt_params = {dimensions[i].name:sr[i] for i in range(len(dimensions))}
opt_params['activation'] = 'relu'
print(opt_params)

In [0]:
# go to parent dir
%cd /content/image-classification/test/
!ls


#Fit the model to extract SavedModel with optimized hyparameters
import image_classifier_tfrecords;reload(image_classifier_tfrecords)
from image_classifier_tfrecords import ImageClassifier
classifier = ImageClassifier(tfrecords_folder=data_dir, use_TPU=use_tpu, 
                                 batch_size=128, transfer_model='Inception')
classifier.fit(**opt_params, save_model=True)

In [0]:
%cd /content/image-classification/data/trained_models/
!ls
# download saved model files
colab.files.download("trained_model.h5")

/content/image-classification/data/trained_models
trained_model.h5
